In [ ]:
%load_ext autoreload
%autoreload 2
from CO2_identify import *
from labeling import *
from mynetwork import CO2mask
from torch.utils.data import DataLoader
from scipy.ndimage import gaussian_filter
from torchvision.transforms.functional import resize
from sklearn.neighbors import NearestNeighbors

import torchvision
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import json
import glob
import shutil

In [ ]:
fn = '../define_path.txt'
with open(fn) as f:
    lines = f.readlines()
for idx, line in enumerate(lines):
    if idx == 1:
        dir_co2 = line.split('=')[1][:-1]
    if idx == 11:
        analpath = line.split('=')[1][:-1]

# Load the testing data and testing network

## Load the original testing data

In [ ]:
# file pathes
blfn = f'{dir_co2}/94p10/2010 processing/data/94p10nea.sgy' # baseline data processed in 2010
tlfn = f'{dir_co2}/10p10/2010 processing/data/10p10nea.sgy' # timelapse (2010) data processed in 2010
mkfn = f'../resources/label/masks.dat' # CO2 masks interpreted from 1994 and 2010 data processed in 2010

In [ ]:
# load the data head
D0 = dataload(fn=blfn)
Dt = dataload(fn=tlfn)

In [ ]:
# get the entire 3D data from the three data volume
d0,xd,yd,td = D0.getdata()
dt,_,_,_ = Dt.getdata()

In [ ]:
# 3d resize class
RS = resize3d(1,D=64,H=64,W=64)

In [ ]:
# readin CO2 mask
masks = np.fromfile(f'{mkfn}',dtype=np.float32)
masks = np.reshape(masks,(D0.nx,D0.ny,D0.nt))

## Select certain part of the original data for testing

### display objective inline section for baseline and time-lapse dataset

In [ ]:
# inline section comparison
InN = 130
fig,ax = plt.subplots(1,2,figsize=(20,5))
ax[0].imshow(d0[InN,:,:].T,aspect=0.5,cmap='gray')
ax[1].imshow(dt[InN,:,:].T,aspect=0.5,cmap='gray')
for i in range(2):
    ax[i].set_xlabel('Xline No.')
    ax[i].set_ylabel('Time (2 ms)')
    if i<1:
        ax[i].set_title(f'1994 baseline, processed in 2010')
    else:
        ax[i].set_title(f'2010 time-lapse, processed in 2010')

### display objective trace for baseline and time-lapse dataset

In [ ]:
XnN = 245
# trace comparison
t1 = 0
t2 = 1000
fig,ax = plt.subplots(1,1,figsize=(15,5))
ax.plot(d0[InN,XnN,:],label=f'baseline processed in 2010')
ax.plot(dt[InN,XnN,:],label=f'time-lapse processed in 2010')
ax.set_xlim(t1,t2)
ax.set_xlabel('Time (2 ms)')
ax.set_ylabel('Amplitude')
ax.legend()

### determine the sampling position for two cube centers

In [ ]:
outpath = f'{analpath}/data'

In [ ]:
Ni,Nx,Nt = 128,128,256
rs = [64,64,64]

In [ ]:
Isample = [InN]
Xsample = [XnN]
Tsample = [450,850]

In [ ]:
# generate parameter csv
pm_info = OrderedDict()
pm_info['data_dim'] = [D0.nx,D0.ny,D0.nt] # [ensemble number,trace number per ensemble,sample number per trace]
pm_info['data_path'] = [blfn,tlfn]
pm_info['patch_osize'] = [Ni,Nx,Nt]
pm_info['patch_nsize'] = rs
pm_info['patch_number'] = 1
pm_info['sample_strategy'] = 1
pm_info['max_0co2'] = 0 # if none, means no limit for zero co2 masks
with open(f'{outpath}/pm_info.json','w') as f:
    f.write(json.dumps(pm_info))
with open(f'{outpath}/pm_info.json','r') as read_file:
    loaded_pm = json.loads(read_file.read())
    print(loaded_pm)

In [ ]:
eps = 1e-5
hNi,hNx,hNt = Ni//2,Nx//2,Nt//2
# sample the patches iteratively
hNi1 = Ni-hNi
hNx1 = Nx-hNx
hNt1 = Nt-hNt
patch_info = []
c = 0
for Is,Xs,Ts in product(Isample,Xsample,Tsample):
    # slice the data
    R0 = d0[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1]
    Rt = dt[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1]
    # slice the mask
    M = np.array(masks[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
    M = RS.resize(torch.tensor(M).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=128,H=128,W=128)
    M.tofile(f'{outpath}/Mask_{c}.dat')
    # normalize R0 and Rt respectively
    Rm0,Rs0 = np.mean(R0),np.std(R0)
    Rmt,Rst = np.mean(Rt),np.std(Rt)
    R0 = (R0-Rm0)/(Rs0+eps)
    Rt = (Rt-Rmt)/(Rst+eps)
    # stack R0 and Rt
    R0t = torch.tensor(np.stack((R0,Rt))).unsqueeze(0)
    R0t = RS.resize(R0t,NP=True) # 5D: (N=1,C=2,D=64,H=64,W=64)
    # save the patches and corresponding masks to outpath
    R0t.tofile(f'{outpath}/R0t_{c}.dat')
    # record the patch information in patch_info dict
    pf = OrderedDict()
    pf['Ptch_id'] = c
    pf['Mask_id'] = c
    pf['ct'] = [Is,Xs,Ts]
    pf['mean'] = [Rm0,Rmt]
    pf['std'] = [Rs0,Rst]
    patch_info.append(pf)
    c += 1
# save the patch info
pd.DataFrame.from_dict(patch_info, orient='columns').to_csv(f'{outpath}/patch_info.csv')

In [ ]:
# load the analysis dataset
pmf = 'pm_info.json'
pdf = 'patch_info.csv'
analy = dataset_patch(outpath,pmf,pdf)
Nana = len(analy)
print(f'Training dataset size: {Nana}')

In [ ]:
# display the analysis dataset
id_list = range(Nana)
psa = patch_show(analy,id_list)

In [ ]:
# revise the show slice index for psa the same
psa.ixtid[0] = [32,32,32]
for i in range(Nana):
    psa.ixtid[i] = psa.ixtid[0]

## Load the testing network

In [ ]:
path_net = f'../resources/NNpred3D/train_fulllabel/co2_identify_best.pt'

In [ ]:
# load the network
networktest = CO2mask()
networktest.load_state_dict(torch.load(path_net,map_location=torch.device('cpu')))
networktest = networktest.eval()

## Original test of the network

In [ ]:
# perform testing
loader_analy = DataLoader(
     analy
    ,batch_size = Nana)
# allocate memory for testing batches
Anpred = np.zeros((Nana,rs[0],rs[1],rs[2]))
for batch in loader_analy:
    R0t, Mr, _ = batch
    # forward modeling
    with torch.no_grad():
        pMask = networktest(R0t)
    # record the sampled testing patches for later display
    Anpred = pMask.squeeze(1).cpu().detach().numpy()
# display the patch fitting in the analysing dataset
psa.view3d(Anpred,rcstride=(1,1))

In [ ]:
Anpred.tofile(f'{outpath}/orgpred.dat')

In [ ]:
R0t_basic = torch.clone(R0t)

In [ ]:
# output original R0 and Rt for display
R0t.numpy().tofile(f'{outpath}/orgR0t.dat')

# Tests:

In [ ]:
def shift3D(d,ts):
    r'''
    dynamically shift d in 3D with 3D t:
        d--3D float array (nx,ny,nt)
        ts--same size as d
    '''
    nx,ny,nt = d.shape
    hnt = (nt+1)//2
    D = np.fft.fft(d,axis=2)
    w = np.fft.fftfreq(nt,1)*2*pi
    w = w[1:hnt]
    t = np.arange(nt)
    W = np.expand_dims(w,0)
    T = np.expand_dims(t,1)
    ds = np.zeros_like(d)
    for i,j in product(range(nx),range(ny)):
        Dij = np.expand_dims(D[i,j,1:hnt],0)
        tsij = np.expand_dims(ts[i,j,:],1)
        F = np.exp(1j*W*(T+tsij))
        ds[i,j,:] = 2*np.sum((Dij*F).real,axis=1)+D[i,j,0].real
    return ds

## Scale up the middle part of the multiple time-lapse image and the baseline image

In [ ]:
# define the path
outpath0 = f'{analpath}/data_s0'

In [ ]:
fn = glob.glob(f'{outpath}/*')
for fni in fn:
    shutil.copy(fni, f'{outpath0}/')

In [ ]:
# define the scalar
s = 4

In [ ]:
I1,I2 = 16,48
X1,X2 = 16,48
T1,T2 = 16,48

In [ ]:
H = hanning3d((I2-I1,X2-X1,T2-T1),(0.2,0.2,0.2))

In [ ]:
S = np.ones(tuple(rs),dtype=np.float32)
S[I1:I2,X1:X2,T1:T2] += H*(s-1)

In [ ]:
show3D(S,xyzi=psa.ixtid[0],rcstride=(2,2))

In [ ]:
S = torch.tensor(S)

In [ ]:
Sb = torch.unsqueeze(torch.unsqueeze(S,0),0)

In [ ]:
r'''
# time shift
ts = np.zeros(tuple(rs),dtype=np.float32)
for i,j in product(range(rs[0]),range(rs[1])):
    ts[i,j,:] = np.cumsum(Sb[0,0,i,j].numpy()-1)*0.5/20
show3D(ts,xyzi=psa.ixtid[0],rcstride=(2,2))
'''

In [ ]:
# scale the time-lapse multiple and the baseline, respectively
# mutliple
R0t[0,0] = R0t_basic[1,0]
R0t[0,1] = R0t_basic[1,1]*Sb[0,0]
#R0t[0,1] = torch.tensor(shift3D(R0t[0,1].numpy(),-ts))
Rm,Rs = torch.mean(R0t[0,1]),torch.std(R0t[0,1])
R0t[0,1] = (R0t[0,1]-Rm)/(Rs+eps)
R0t[0].numpy().tofile(f'{outpath0}/R0t_{0}.dat')
# baseline
R0t[1,1] = R0t_basic[1,0]*Sb[0,0]
#R0t[1,1] = torch.tensor(shift3D(R0t[1,1].numpy(),-ts))
Rm,Rs = torch.mean(R0t[1,1]),torch.std(R0t[1,1])
R0t[1,1] = (R0t[1,1]-Rm)/(Rs+eps)
R0t[1].numpy().tofile(f'{outpath0}/R0t_{1}.dat')

In [ ]:
# output S for display
S.numpy().tofile(f'{outpath0}/scale.dat')

In [ ]:
# output scaled Rt for display
R0t.numpy().tofile(f'{outpath}/testR0t.dat')

## Reperform the testing on the new dataset

In [ ]:
# reload the analysis dataset
analy = dataset_patch(outpath0,pmf,pdf)
Nana = len(analy)
print(f'Training dataset size: {Nana}')

In [ ]:
# parameters for display the analysis dataset
id_list = range(Nana)
psa = patch_show(analy,id_list)
# revise the show slice index for psa the same
psa.ixtid[0] = [32,32,32]
for i in range(Nana):
    psa.ixtid[i] = psa.ixtid[0]

In [ ]:
# perform testing
loader_analy = DataLoader(
     analy
    ,batch_size = Nana)
# allocate memory for testing batches
Anpred = np.zeros((Nana,rs[0],rs[1],rs[2]))
for batch in loader_analy:
    R0t, Mr, _ = batch
    # forward modeling
    with torch.no_grad():
        pMask = networktest(R0t)
    # record the sampled testing patches for later display
    Anpred = pMask.squeeze(1).cpu().detach().numpy()
# display the patch fitting in the analysing dataset
psa.view3d(Anpred,rcstride=(1,1))

In [ ]:
Anpred.tofile(f'{outpath0}/testpred.dat')

In [ ]:
# trace comparison
I,X,_ = psa.ixtid[0]
ylim = [-6,6]
fig,ax = plt.subplots(1,2,figsize=(20,5))
ax[0].plot(R0t[0,0,I,X,:],label=f'94p10, primary')
ax[0].plot(R0t[0,1,I,X,:],label=f'10p10, primary')
ax[1].plot(R0t[1,0,I,X,:],label=f'94p10, multiple')
ax[1].plot(R0t[1,1,I,X,:],label=f'10p10, multiple')
for i in range(2):
    ax[i].set_xlim(0,rs[2])
    ax[i].set_ylim(ylim[0],ylim[1])
    ax[i].set_xlabel('Time (2 ms)')
    ax[i].set_ylabel('Amplitude')
    ax[i].legend()